<a href="https://colab.research.google.com/github/ninenine-9/legaldetainment/blob/main/fewshot_COT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pathway 1: Advanced Prompting with Few-Shot Chain-of-Thought (CoT)

This approach focuses on refining how you ask the model to perform the coding task. Instead of giving the model all codes at once, we will break the problem down into a series of simpler decisions. This method requires no model training and can yield dramatic improvements in accuracy by reducing the model's cognitive load.

Your current script likely iterates through participants and asks a model to assign all relevant codes in one go. We will modify this to iterate through each participant and then each potential code, turning a complex multi-label task into a series of focused binary (APPLIES / DOES NOT APPLY) classifications.

In [ ]:
import pandas as pd
import time
import os
from google.colab import drive

# Install required libraries
!pip install -q -U transformers bitsandbytes accelerate

# Import libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import json

print("✔️ Imports completed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.1 MB/s eta 0:00:00
✔️ Imports completed


## Model loading

In [ ]:
# --- Load the LLM and Tokenizer ---

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Configure quantization to load the model in 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"  # Automatically maps the model to the available GPU
)

print("✔️ LLM and Tokenizer loaded successfully.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✔️ LLM and Tokenizer loaded successfully.


## Data

In [ ]:
# Data retrieval

%cd /content

!git clone https://github.com/ninenine-9/legaldetainment.git

drive.mount('/content/drive', force_remount=True)

!cp -r /content/legaldetainment /content/drive/MyDrive/

print("✔️ Git Repository successfully cloned")

with open("/content/legaldetainment/INPUTS/legaldetainment_story.md", "r", encoding="utf-8") as f:
    study_context = f.read()

with open("/content/legaldetainment/INPUTS/legaldetainment_codinginstructions.md", "r", encoding="utf-8") as f:
    coding_manual = f.read()

# Data loading

blank_df = pd.read_excel("/content/legaldetainment/DATA/legaldetainment_blankdata.xlsx", sheet_name = '30participants') # 📋 for cell H & G

blank_df = blank_df.dropna(subset=["Pno"])
blank_df.columns = blank_df.columns.str.strip()

blank_df = blank_df[["Pno", "Qual", "GuiltMoreLikely",
        "GuiltLessLikely",
        "NoInformation_Evidence",
        "InnocentUntilProvenGuilty",
        "Confound"]]

print(f"Dataset size: {blank_df.shape}")

humandata = pd.read_excel("/content/legaldetainment/DATA/legaldetainment_humandata.xlsx", sheet_name = '30participants') # 📋 for cell I

humandata = humandata.dropna(subset=["Pno"])
humandata = humandata.replace({0: "No", 1: "Yes", "0": "No", "1": "Yes"})
humandata = humandata.rename(columns={
    "NoInformation/Evidence": "NoInformation_Evidence",
    "Unclassified/Other": "Unclassified_Other"
})
humandata.columns = humandata.columns.str.strip()

humandata = humandata[["Pno", "Qual", "GuiltMoreLikely",
        "GuiltLessLikely",
        "NoInformation_Evidence",
        "InnocentUntilProvenGuilty",
        "Confound"]]

print("✔️ Data loading done")

/content
fatal: destination path 'legaldetainment' already exists and is not an empty directory.
Mounted at /content/drive
✔️ Git Repository successfully cloned
Dataset size: (30, 7)
✔️ Data loading done


## Codebook

In [ ]:
# LLM-Adapted Codebook
# This structured list of dictionaries translates your human-centric coding instructions into a
# machine-readable format. It is the foundation for building high-performance prompts.

llm_codebook = [
    {
        "Code_Name": "GuiltMoreLikely",
        "Simple_Definition": "This code applies when the participant concludes the defendant is more likely to be guilty because they infer the judge's decision to *deny bail* was based on secret, negative information (like a criminal history).",
        "Inclusion_Criteria": """- The text must connect the judge's decision to **deny bail** (or keep the defendant detained) to a higher likelihood of guilt.
- The text must state or imply that the judge's decision is based on extra information that the participant does not have (e.g., 'the judge must know something,' 'he probably has a record').""",
        "Exclusion_Criteria": """- DO NOT apply if the participant mentions the judge *granting* bail.
- DO NOT apply if the participant simply says the defendant is guilty without linking it to the judge's bail decision.
- DO NOT apply if the participant says there is not enough information to make a judgment (this would be NoInformation_Evidence).
- DO NOT apply if the participant's reasoning is about the judge being biased (this would be Confound).""",
        "Positive_Exemplar_Text": "The judge wouldn't have kept him in jail if there wasn't something else going on. He probably has a criminal history, so I think he's more likely to be guilty.",
        "Positive_Exemplar_Explanation": "This meets all criteria: it mentions detainment, infers the judge knows about a 'criminal history,' and concludes guilt is more likely.",
        "Negative_Exemplar_Text": "He was denied bail, so the judge is probably biased against him now.",
        "Negative_Exemplar_Explanation": "This is a hard negative. It mentions bail denial but focuses on future judge bias, which fits the `Confound` code, not the inference of secret evidence required for `GuiltMoreLikely`."
    },
    {
        "Code_Name": "GuiltLessLikely",
        "Simple_Definition": "This code applies when the participant concludes the defendant is less likely to be guilty because they infer the judge's decision to *grant bail* was based on secret, positive information (like a solid alibi).",
        "Inclusion_Criteria": """- The text must connect the judge's decision to **grant bail** (or release the defendant) to a lower likelihood of guilt.
- The text must state or imply that the judge's decision is based on extra information that the participant does not have (e.g., 'the judge must have seen the alibi').""",
        "Exclusion_Criteria": """- DO NOT apply if the participant mentions the judge *denying* bail.
- DO NOT apply if the participant simply says the defendant is innocent without linking it to the judge's bail decision.
- DO NOT apply if the participant says there is not enough information to make a judgment (this would be NoInformation_Evidence).""",
        "Positive_Exemplar_Text": "If the judge let him go, he must have a solid alibi or something. I'd say he's less likely to be guilty.",
        "Positive_Exemplar_Explanation": "This meets all criteria: it mentions release, infers the judge knows about a 'solid alibi,' and concludes guilt is less likely.",
        "Negative_Exemplar_Text": "It's good he got bail, but we should remember he is innocent until proven guilty anyway.",
        "Negative_Exemplar_Explanation": "This is a hard negative. Although it mentions bail, the core reasoning invokes the legal principle of `InnocentUntilProvenGuilty`, which is a different code."
    },
    {
        "Code_Name": "NoInformation_Evidence",
        "Simple_Definition": "This code applies when the participant explicitly states that they cannot make a judgment because there is not enough information or evidence provided.",
        "Inclusion_Criteria": """- The text must contain phrases that explicitly state a lack of information or evidence, such as 'not enough information,' 'no evidence,' 'can't decide based on this,' or 'need more facts.'""",
        "Exclusion_Criteria": """- DO NOT apply if the participant makes a judgment about guilt or innocence, even if they express some uncertainty (e.g., 'I guess he's guilty, but it's hard to say').
- DO NOT apply if the participant is simply confused about the question itself; the focus must be on the lack of case evidence.""",
        "Positive_Exemplar_Text": "Based on this, I can't say. There's not enough evidence here to make a call.",
        "Positive_Exemplar_Explanation": "This is a direct statement about the lack of evidence preventing a judgment.",
        "Negative_Exemplar_Text": "I lean towards him being guilty, but I don't really have enough information.",
        "Negative_Exemplar_Explanation": "This is a hard negative because even though the participant mentions a lack of information, they still make a judgment ('lean towards him being guilty'), so this code does not apply."
    },
    {
        "Code_Name": "InnocentUntilProvenGuilty",
        "Simple_Definition": "This code applies when the participant references the legal principle that a person is considered innocent until their guilt is proven in court, regardless of other factors.",
        "Inclusion_Criteria": """- The text must explicitly use the phrase 'innocent until proven guilty' or a very close variation (e.g., 'innocent till guilty').
- OR, the text must clearly express the same sentiment, such as 'we shouldn't prejudge him,' or 'none of this matters until the trial proves he did it.'""",
        "Exclusion_Criteria": """- DO NOT apply if the participant simply says the defendant is 'innocent' without referencing the legal principle or the idea of waiting for proof at trial.""",
        "Positive_Exemplar_Text": "It doesn't matter what the judge decided about bail, he is innocent until proven guilty.",
        "Positive_Exemplar_Explanation": "This is a perfect example as it uses the exact key phrase to state the legal principle.",
        "Negative_Exemplar_Text": "I think he's innocent.",
        "Negative_Exemplar_Explanation": "This is a hard negative because it states an opinion on innocence but does not invoke the specific legal principle of 'innocent until proven guilty'."
    },
    {
        "Code_Name": "Confound",
        "Simple_Definition": "This code applies when the participant expresses concern about the fairness of the main trial because the same judge will be involved and may now be biased due to the bail decision.",
        "Inclusion_Criteria": """- The text must state that the **same judge** from the bail hearing will also preside over the main trial.
- AND/OR the text must state that the judge might now be **biased** (either for or against the defendant) as a direct result of the bail decision.""",
        "Exclusion_Criteria": """- DO NOT apply for general statements about the legal system being unfair. The comment must be specific to the judge's potential bias from this specific situation.
- DO NOT apply if the participant thinks the judge has secret information but doesn't mention bias (this would be `GuiltMoreLikely` or `GuiltLessLikely`).""",
        "Positive_Exemplar_Text": "Since it's the same judge for the trial, he might be biased against the defendant now that he's denied him bail.",
        "Positive_Exemplar_Explanation": "This meets both potential criteria: it mentions the 'same judge' and explicitly states the risk of bias.",
        "Negative_Exemplar_Text": "Because each judge is new to the case they will have fresh eyes",
        "Negative_Exemplar_Explanation": "This is a hard negative because it states an opinion on innocence but does not invoke the specific legal principle of 'innocent until proven guilty'."
    }
]

## Prompt generation

In [ ]:
def construct_high_performance_prompt(text_to_analyze, code_entry):
    """
    Constructs a detailed, few-shot prompt with Chain-of-Thought examples
    for a single code classification task.

    Args:
        text_to_analyze (str): The participant's text to be coded.
        code_entry (dict): A dictionary from the llm_codebook for a single code.

    Returns:
        str: A fully formatted prompt ready for the LLM.
    """

    # Format the criteria lists into bulleted strings for clarity in the prompt
    inclusion_criteria_str = "\n".join([f"- {item}" for item in code_entry['Inclusion_Criteria']])
    exclusion_criteria_str = "\n".join([f"- {item}" for item in code_entry['Exclusion_Criteria']])

    # --- Construct the Chain-of-Thought reasoning for the few-shot examples ---
    # This is where we "teach" the model how to think by showing our work.

    # Reasoning for the positive example
    positive_reasoning = (
        f"Step 1: The code '{code_entry['Code_Name']}' is about '{code_entry}'. "
        f"Step 2: The provided text is '{code_entry}'. "
        f"Step 3: I will evaluate this against the criteria. The text meets the inclusion criteria because {code_entry['Positive_Exemplar_Explanation']}. "
        f"Step 4: Therefore, the code applies."
    )

    # Reasoning for the negative example
    negative_reasoning = (
        f"Step 1: The code '{code_entry['Code_Name']}' is about '{code_entry}'. "
        f"Step 2: The provided text is '{code_entry}'. "
        f"Step 3: I will evaluate this against the criteria. The text does not meet the inclusion criteria because {code_entry['Negative_Exemplar_Explanation']}. "
        f"Step 4: Therefore, the code does not apply."
    )

    # --- Assemble the final prompt using an f-string and XML tags for structure ---
    # Using XML tags helps the model differentiate between instructions, examples, and the task.[3]
    prompt = f"""
<Role_and_Task_Definition>
You are an expert qualitative research analyst. Your task is to determine if the code '{code_entry['Code_Name']}' should be applied to the text provided in the <Text_to_Analyze> section.
You must first provide a step-by-step reasoning for your decision within the 'reasoning' field of a JSON object. After your reasoning, you must make a final binary decision ('APPLIES' or 'DOES NOT APPLY') in the 'decision' field.
</Role_and_Task_Definition>

<Codebook_Information>
<Code_Name>{code_entry['Code_Name']}</Code_Name>
<Simple_Definition>{code_entry}</Simple_Definition>
<Inclusion_Criteria>
{inclusion_criteria_str}
</Inclusion_Criteria>
<Exclusion_Criteria>
{exclusion_criteria_str}
</Exclusion_Criteria>
</Codebook_Information>

<Few-Shot_Exemplars>
<Positive_Example>
<Input_Text>{code_entry}</Input_Text>
<Output>
{{
  "reasoning": "{positive_reasoning}",
  "decision": "APPLIES"
}}
</Output>
</Positive_Example>

<Negative_Example>
<Input_Text>{code_entry}</Input_Text>
<Output>
{{
  "reasoning": "{negative_reasoning}",
  "decision": "DOES NOT APPLY"
}}
</Output>
</Negative_Example>
</Few-Shot_Exemplars>

<Input_Data_and_Output_Schema>
<Text_to_Analyze>
{text_to_analyze}
</Text_to_Analyze>
<Final_Output>
Provide your final answer in a single, valid JSON object using the exact format shown in the examples above.
</Final_Output>
</Input_Data_and_Output_Schema>
"""
    return prompt

## LLM Inferencing

In [ ]:
def get_llm_classification(prompt, model, tokenizer):
    """
    Sends a prompt to the LLM and gets a classification decision.

    Args:
        prompt (str): The detailed prompt for the model.
        model: The loaded Hugging Face model.
        tokenizer: The loaded Hugging Face tokenizer.

    Returns:
        dict: A dictionary containing the 'decision' and 'reasoning', or an error.
    """
    # Tokenize the prompt and send it to the GPU
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Get the length of the input tokens
    input_length = inputs.input_ids.shape[-1]

    # Generate a response from the model
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,  # Limit the length of the generated response
        temperature=0.1      # Use a low temperature for more deterministic, less creative output
    )

    # Decode the output and skip the prompt part
    # Extract only the newly generated tokens after the input prompt
    response_text = tokenizer.decode(outputs[0, input_length:], skip_special_tokens=True)


    # --- Robust JSON Parsing ---
    # The model's output is a string; we need to find and parse the JSON within it.
    try:
        # Find the start of the JSON object
        json_start_index = response_text.find('{')
        # Find the end of the JSON object
        json_end_index = response_text.rfind('}') + 1

        if json_start_index!= -1 and json_end_index!= -1:
            json_string = response_text[json_start_index:json_end_index]
            return json.loads(json_string)
        else:
            return {"decision": "PARSE_ERROR", "reasoning": response_text}
    except json.JSONDecodeError:
        return {"decision": "JSON_DECODE_ERROR", "reasoning": response_text}

## Processing

## Saving outputs

In [ ]:
# p_amount = int(len(data))

# csv_name = (f"{p_amount}_{model_input[:4]}")

output_path = f"/content/legaldetainment/RESULTS/COTv1.csv" # 💾

output_df = pd.DataFrame(results)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
output_df.to_csv(output_path, index=False)
print(f"✔️ Results saved to {output_path}")

## Evaluation

!! Need to translate Applies/0 to Yes/No

In [ ]:
cluster_codes = ["GuiltMoreLikely", "GuiltLessLikely", "NoInformation_Evidence"]
independent_codes = ["InnocentUntilProvenGuilty", "Confound"]
all_codes = cluster_codes + independent_codes

In [ ]:


# Assume 'humandata' is your ground truth DataFrame
# Assume 'blank_df' is the data to be coded
# Assume 'llm_codebook' is a list of dictionaries, one for each code, created in Step 1

results = []
p_amount = len(blank_df)

# --- Main Processing Loop ---
start_time = time.time()

for index, row in blank_df.iterrows():
    participant_id = row['Pno']
    text_to_analyze = row['Qual'] # Replace with your actual text column name

    for code_entry in llm_codebook:
        code_name = code_entry['Code_Name']

        # 1. Construct the detailed prompt for this specific code
        prompt = construct_high_performance_prompt(text_to_analyze, code_entry)
        print("✔️ Prompt built")

        # 2. Get the LLM's decision for this single code
        # This function would contain your model-loading and generation logic
        llm_output_json = get_llm_classification(prompt, model, tokenizer)
        print("✔️ Decision made by LLM")

        # 3. Get the ground truth for comparison
        ground_truth_row = humandata[humandata['Pno'] == participant_id]
        gold_standard_decision = "APPLIES" if ground_truth_row[code_name].iloc == 1 else "DOES NOT APPLY"
        print("✔️ LLM decision compared to human")


        # 4. Store results
        results.append({
            'Pno': participant_id,
            'code_name': code_name,
            'llm_decision_raw': llm_output_json.get('decision'),
            'llm_decision': 1 if llm_output_json.get('decision') == "APPLIES" else 0,
            'human_code': 1 if gold_standard_decision == "APPLIES" else 0,
            'llm_reasoning': llm_output_json.get('reasoning')
        })

end_time = time.time()
elapsed = end_time - start_time
print(f"\n✔️ Classification completed in {elapsed:.2f} seconds.")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✔️ Prompt built


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✔️ Decision made by LLM
✔️ LLM decision compared to human
✔️ Prompt built


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✔️ Decision made by LLM
✔️ LLM decision compared to human
✔️ Prompt built


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✔️ Decision made by LLM
✔️ LLM decision compared to human
✔️ Prompt built


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os
from google.colab import files

print(f"Found {len(results)} results in memory. Saving them now...")

# --- Define the output path ---
# Use the same naming convention you had before
p_amount = 30 # Or the total number of participants you started with
conf_input = "0.5" # Replace with the confidence you used
model_input = "mistral" # Replace with the model name you used

csv_name = (f"{p_amount}_partial_CF{conf_input}_{model_input[:4]}")
output_path = f"/content/legaldetainment/RESULTS/{csv_name}.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# --- Convert the list to a DataFrame and save ---
if results:
    output_df = pd.DataFrame(results)
    output_df.to_csv(output_path, index=False)
    print(f"✅ Successfully saved {len(output_df)} rows to {output_path}")

    # --- Download the file immediately to your local computer ---
    print("Downloading the file to your computer as a backup...")
    files.download(output_path)
else:
    print("The 'results' list is empty. No data was saved.")

Found 72 results in memory. Saving them now...
✅ Successfully saved 72 rows to /content/legaldetainment/RESULTS/30_partial_CF0.5_mist.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import pandas as pd

# --- IMPORTANT: Replace this with the actual path to your saved CSV file ---
csv_path = "/content/legaldetainment/RESULTS/30_partial_CF0.5_mist.csv"

try:
    # Read the CSV file into a pandas DataFrame
    results_df = pd.read_csv(csv_path)

    # Print the shape of the DataFrame (rows, columns)
    print(f"DataFrame shape: {results_df.shape}\n")

    # Print the column names
    print("Columns:")
    print(results_df.columns.tolist())
    print("\n" + "="*50 + "\n")


    # Print the first 5 rows to show the structure
    print("First 5 rows of the DataFrame:")
    display(results_df.head())

except FileNotFoundError:
    print(f"ERROR: File not found at '{csv_path}'. Please double-check the file name and path.")
except Exception as e:
    print(f"An error occurred: {e}")

DataFrame shape: (72, 6)

Columns:
['Pno', 'code_name', 'llm_decision_raw', 'llm_decision', 'human_code', 'llm_reasoning']


First 5 rows of the DataFrame:


,Pno,code_name,llm_decision_raw,llm_decision,human_code,llm_reasoning
0,232,GuiltMoreLikely,JSON_DECODE_ERROR,0,0,</Text_to_Analyze>\n</Input_Data_and_Output_Sc...
1,232,GuiltLessLikely,JSON_DECODE_ERROR,0,0,\n<Text_to_Analyze>\nI think he has a good cha...
2,232,NoInformation_Evidence,JSON_DECODE_ERROR,0,0,\n<Text_to_Analyze>\nI think he has a good cha...
3,232,InnocentUntilProvenGuilty,PARSE_ERROR,0,0,\n<Text_to_Analyze>\nI think he's innocent.\n<...
4,232,Confound,JSON_DECODE_ERROR,0,0,\n<Text_to_Analyze>\nI think he has a good cha...


In [15]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# --- Load Your Saved Data ---
# Make sure to update this path to your actual CSV file
csv_path = "/content/legaldetainment/RESULTS/30_partial_CF0.5_mist.csv"
results_df = pd.read_csv(csv_path)

print(f"Loaded {len(results_df)} total classifications.")

# --- 1. Filter out the rows where the model failed to produce valid JSON ---
# Corrected the isin() method to include the list of values to exclude
successful_results_df = results_df[~results_df['llm_decision_raw'].isin(['PARSE_ERROR', 'JSON_DECODE_ERROR'])].copy()
error_count = len(results_df) - len(successful_results_df)
print(f"Found {error_count} errors ({error_count/len(results_df):.1%}). Analyzing {len(successful_results_df)} successful classifications.\n")

# Convert 'APPLIES'/'DOES NOT APPLY' to 1/0 for the model's decision
successful_results_df['llm_decision_numeric'] = successful_results_df['llm_decision_raw'].apply(lambda x: 1 if x == 'APPLIES' else 0)


# --- 2. Reshape the data from "long" to "wide" format ---
# This pivots the data so each row is one participant and each column is a code, matching your humandata.
model_coded_wide_df = successful_results_df.pivot(index='Pno', columns='code_name', values='llm_decision_numeric')
model_coded_wide_df.reset_index(inplace=True)
model_coded_wide_df = model_coded_wide_df.rename_axis(None, axis=1)

print("Model results reshaped into wide format:")
display(model_coded_wide_df.head())


# --- 3. Merge with Human Data and Evaluate ---
# Make sure your 'humandata' DataFrame is loaded and formatted (1s and 0s)
# This assumes 'humandata' has 1/0 for codes, not "Yes"/"No"
humandata_numeric = humandata.replace({"Yes": 1, "No": 0})

# Merge the dataframes on the participant number
eval_df = pd.merge(
    humandata_numeric,
    model_coded_wide_df,
    on='Pno',
    suffixes=('_human', '_model')
)

print("\nMerged data for evaluation:")
display(eval_df.head())


# --- 4. Calculate and Print Performance Metrics ---
all_human_labels = []
all_model_labels = []

# Get a list of the code columns we are evaluating
code_columns = [col for col in model_coded_wide_df.columns if col!= 'Pno']

for code in code_columns:
    human_col = f"{code}_human"
    model_col = f"{code}_model"

    # Ensure columns exist before trying to access them
    if human_col in eval_df.columns and model_col in eval_df.columns:
        # Drop rows where the model didn't have a prediction (NaN)
        valid_evals = eval_df[[human_col, model_col]].dropna()

        all_human_labels.extend(valid_evals[human_col].tolist())
        all_model_labels.extend(valid_evals[model_col].tolist())

print("\n" + "="*60)
print("Overall Performance Metrics (on successfully decoded results):")
print("="*60)

# Calculate overall metrics
precision, recall, f1, _ = precision_recall_fscore_support(
    all_human_labels,
    all_model_labels,
    average='binary', # Use 'binary' for 1/0 labels
    pos_label=1,
    zero_division=0
)

print(f"Overall Precision: {precision:.3f}")
print(f"Overall Recall:    {recall:.3f}")
print(f"Overall F1-Score:  {f1:.3f}")
print("="*60)

Loaded 72 total classifications.
Found 72 errors (100.0%). Analyzing 0 successful classifications.

Model results reshaped into wide format:


,Pno



Merged data for evaluation:


/tmp/ipython-input-4119992410.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  humandata_numeric = humandata.replace({"Yes": 1, "No": 0})


,Pno,Qual,GuiltMoreLikely,GuiltLessLikely,NoInformation_Evidence,InnocentUntilProvenGuilty,Confound



Overall Performance Metrics (on successfully decoded results):
Overall Precision: 0.000
Overall Recall:    0.000
Overall F1-Score:  0.000


### 🤖 Of LM vs human

In [ ]:
# Compare human and model coding
eval_df = humandata[["Pno"] + all_codes].merge(
    output_df[["Pno"] + all_codes],
    on="Pno",
    how="inner"
)

# Boolean: all codes match for that Pno
eval_df["all_match"] = eval_df.apply(
    lambda row: all(row[code + "_x"] == row[code + "_y"] for code in all_codes),
    axis=1
)

# Count matches
total = len(eval_df)
matches = eval_df["all_match"].sum()
percentage = matches / total * 100 if total else 0

print(f"\nTotal Pnos compared: {total}")
print(f"Matching on all codes: {matches} ({percentage:.1f}%)") # 📋 for cell N & O

# Print Pnos that match on all codes
matching_pnos = eval_df[eval_df['all_match'] == True]['Pno'].tolist()
print(f"Pnos matching on all codes: {matching_pnos}") # 📋 for cell P

### 🚀 Of LM performance

In [ ]:
import sys
from sklearn.metrics import precision_recall_fscore_support

# 🔸 Redirect stdout to file
log_file = f"/content/legaldetainment/RESULTS/COTv1_log.txt"
original_stdout = sys.stdout
with open(log_file, "w", encoding="utf-8") as f:
    sys.stdout = f

    print("\n" + "="*60)
    print("Per-Code Metrics:")
    print("="*60)

    for code in all_codes:
        human_col = f"{code}_x"
        model_col = f"{code}_y"

        human_vals = eval_df[human_col]
        model_vals = eval_df[model_col]

        precision, recall, f1, support = precision_recall_fscore_support(
            human_vals, model_vals,
            labels=["Yes", "No"],
            average=None,
            zero_division=0
        )

        precision_yes = precision[0]
        recall_yes = recall[0]
        f1_yes = f1[0]
        support_yes = support[0]

        print(f"\n{code}:")
        print(f"  Precision: {precision_yes:.3f}")
        print(f"  Recall:    {recall_yes:.3f}")
        print(f"  F1-Score:  {f1_yes:.3f}")
        print(f"  Support:   {support_yes} actual 'Yes' cases")

    print("\n" + "="*60)
    print("Overall Metrics:")
    print("="*60)

    all_human = []
    all_model = []
    for code in all_codes:
        all_human.extend(eval_df[f"{code}_x"].tolist())
        all_model.extend(eval_df[f"{code}_y"].tolist())

    precision, recall, f1, support = precision_recall_fscore_support(
        all_human, all_model,
        labels=["Yes", "No"],
        average=None,
        zero_division=0
    )

    print(f"Overall Precision: {precision[0]:.3f}")
    print(f"Overall Recall:    {recall[0]:.3f}")
    print(f"Overall F1-Score:  {f1[0]:.3f}")

    # ✅ restore stdout automatically when file closes
    sys.stdout = original_stdout

print(f"Console output saved to: {log_file}")
